In [5]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs


from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
df = pd.read_csv("DDoS-TrainTest.csv")

In [3]:
df2 = pd.read_csv("DDoS-Validation.csv")

In [6]:
df = df.replace('infinity', 2147483645) #string values of infinity where present this switches them with the max float 
df = df.replace('Infinity', 2147483645)
df = df.replace([np.inf, -np.inf], 2147483645) #remove inf values,


df2 = df2.replace('infinity', 2147483645) #string values of infinity where present this switches them with the max float 
df2 = df2.replace('Infinity', 2147483645)
df2 = df2.replace([np.inf, -np.inf], 2147483645) #remove inf values,


In [7]:
dfTrainTestLable = df[["Label2"]]
dfValidationLable = df2[["Label2"]]

In [8]:
df = df.drop('Label2', axis=1)
df2 = df2.drop('Label2', axis=1)

In [9]:
pd.set_option('display.max_rows', None)


Unnamed: 0                      0
 ACK Flag Count                 0
 Active Max                     0
 Active Min                     0
 Active Std                     0
 Average Packet Size            0
 Avg Bwd Segment Size           0
 Avg Fwd Segment Size           0
 Bwd Avg Bytes/Bulk             0
 Bwd Avg Packets/Bulk           0
 Bwd Header Length              0
 Bwd IAT Max                    0
 Bwd IAT Mean                   0
 Bwd IAT Min                    0
 Bwd IAT Std                    0
 Bwd PSH Flags                  0
 Bwd Packet Length Mean         0
 Bwd Packet Length Min          0
 Bwd Packet Length Std          0
 Bwd Packets/s                  0
 Bwd URG Flags                  0
 CWE Flag Count                 0
 Destination Port               0
 Down/Up Ratio                  0
 ECE Flag Count                 0
 Flow Duration                  0
 Flow IAT Max                   0
 Flow IAT Mean                  0
 Flow IAT Min                   0
 Flow IAT Std 

In [10]:
def ForwardFeatureSelecton(model, train_x, train_y):
    sfs = SFS(model,
           k_features=(3,11),
           forward=True,
           floating=False,
           scoring = 'accuracy',
           cv = 5)
    
    sfs.fit(train_x, train_y)
    
    return sfs

In [11]:
def BackwardFeatureSelecton(model, train_x, train_y):
    sbs = SFS(model,
          k_features=(3,11),
          forward=False,
          floating=False,
          scoring = 'accuracy',
          cv = 5)
    sbs.fit(train_x, train_y)
    return sbs

In [12]:
def findbestfeatures(fs):
    score = fs.k_score_
    features = fs.k_feature_idx_
    print('best combination (ACC: %.3f): %s\n' % (fs.k_score_, fs.k_feature_idx_))
    plot_sfs(fs.get_metric_dict(), kind='std_err');
    ret = [score, features]
    return ret

In [13]:
def GetFeatureNames(features, columns):
    ret = []
    for x in range(0, len(features)):
        temp = columns[features[x]]
        ret.append(temp)
    return ret
        

In [14]:
def TrainTest(model, X_train_sfs, X_test_sfs, train_y, test_y):
    model.fit(X_train_sfs, train_y)
    y_pred = model.predict(X_test_sfs)
    return y_pred

In [15]:
def PrintResults(test_y, y_pred):
    print("Accuracy: {0:.2f}%".format(100*accuracy_score(y_pred, test_y)))
    print(" ")
    #Display the Confusion Matrix and Classification Report 
    print("Confusion Matrix:")
    print(confusion_matrix(y_pred, test_y))
    print(" ")
    print("Classification Report:")
    print(classification_report(y_pred, test_y))

In [18]:
def FeatureSelection(model, df, df2):
    train_x, test_x, train_y, test_y = train_test_split(df.drop(' Label', axis =1 ),df[' Label'],test_size=0.3, random_state=42)
    
    train_x2, test_x2, train_y2, test_y2 = train_test_split(df2.drop(' Label', axis =1 ),df2[' Label'],test_size=0.3, random_state=42)
    
    ret = {}
    print("Forward Selection")
    
    RFCsfs = ForwardFeatureSelecton(model, train_x, train_y)
    sfsval = findbestfeatures(RFCsfs)
    sfsNames = GetFeatureNames(sfsval[1], df.columns)
    
    
    
    X_train_sfs = RFCsfs.transform(train_x)
    X_test_sfs = RFCsfs.transform(test_x)
    y_predf = TrainTest(model, X_train_sfs, X_test_sfs, train_y, test_y)
    PrintResults(test_y, y_predf)
    acc = accuracy_score(test_y, y_predf)
    
    X_train_sfs2 = RFCsfs.transform(train_x2)
    X_test_sfs2 = RFCsfs.transform(test_x2)
    y_predf2 = TrainTest(model, X_train_sfs2, X_test_sfs2, train_y2, test_y2)
    PrintResults(test_y2, y_predf2)
    acc2 = accuracy_score(test_y2, y_predf2)
    
    
    ret['Forward Selection Features'] = sfsNames
    ret['Forward Selection Features Index'] = sfsval[1]
    ret['Forward Selection Training Accuracy'] = sfsval[0]
    ret['Forward Selection Test Accuracy'] = acc
    ret['Forward Selection Validation Accuracy'] = acc2
    
    print("Backward Selection")
    RFCsbs = BackwardFeatureSelecton(model, train_x, train_y)
    sbsval = findbestfeatures(RFCsbs)
    sbsNames = GetFeatureNames(sbsval[1], df.columns)

    X_train_sbs = RFCsbs.transform(train_x)
    X_test_sbs = RFCsbs.transform(test_x)
    y_predb = TrainTest(model, X_train_sbs, X_test_sbs, train_y, test_y)
    PrintResults(test_y, y_predb)
    acc = accuracy_score(test_y, y_predb)
    
    
    X_train_sbs2 = RFCsbs.transform(train_x2)
    X_test_sbs2 = RFCsbs.transform(test_x2)
    y_predb2 = TrainTest(model, X_train_sbs2, X_test_sbs2, train_y2, test_y2)
    PrintResults(test_y2, y_predb2)
    acc2 = accuracy_score(test_y2, y_predb2)
    
    ret['Backward Selection Features'] = sbsNames
    ret['Backward Selection Features Index'] = sbsval[1]
    ret['Backward Selection Training Accuracy'] = sbsval[0]
    ret['Backward Selection Test Accuracy'] = acc
    ret['Backward Selection Validation Accuracy'] = acc2
    
    return ret, y_predf, y_predb, y_predf2, y_predb2

In [ ]:
RFC = RandomForestClassifier()
print("Random Forest Classifier")
RFCdf, RCFP1, RCFP2, RCFP3, RCFP4 = FeatureSelection(RFC,df, df2)
RFCdf["Model"] = "Random Forest Classifier"

Random Forest Classifier
Forward Selection


In [ ]:
#(1, 5, 19, 22, 24, 36, 41, 47, 76)